In [11]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from itertools import product
import warnings

In [12]:
import sys, os

sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2

from modules.config import *
from modules.storage import get_results_df, get_demand_model_data
from modules.neural_network import execute_stage, get_first_stage_hyperparameters,get_second_stage_hyperparameters, get_third_stage_hyperparameters

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
def get_model_data(h3_res, time_interval_length):
	model_data = get_demand_model_data(h3_res, time_interval_length)
	# return model_data.sample(frac=0.1)
	return model_data

In [14]:
execute_stage(
    get_model_data,
    NN_FIRST_STAGE_RESULTS_PATH,
    get_first_stage_hyperparameters,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
)

  0%|          | 0/7 [00:00<?, ?it/s]

[17:39:41] batch_size: 8 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained
[17:39:41] batch_size: 16 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained
[17:39:41] batch_size: 32 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained
[17:39:41] batch_size: 64 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained
[17:39:41] batch_size: 128 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained
[17:39:41] batch_size: 256 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained
[17:39:41] batch_size: 512 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained


In [15]:
results = get_results_df(NN_FIRST_STAGE_RESULTS_PATH)

best_batch_size = (
    results[
        (results["h3_res"] == TUNE_H3_RESOLUTION)
        & (results["time_interval_length"] == TUNE_TIME_INTERVAL_LENGTH)
    ]
    .sort_values(by="val_mape", ascending=True)["batch_size"]
    .iloc[0]
)

first_stage_hyperparameters = get_first_stage_hyperparameters()
batch_sizes = list(map(lambda x: x['batch_size'], first_stage_hyperparameters))
max_batch_size = max(batch_sizes)
min_batch_size = min(batch_sizes)

print(f"best batch_size: **{best_batch_size}** - min: {min_batch_size} - max: {max_batch_size}")


best batch_size: **256** - min: 8 - max: 512


In [16]:
get_hyperparameters = lambda : get_second_stage_hyperparameters(best_batch_size)
execute_stage(
    get_model_data,
    NN_SECOND_STAGE_RESULTS_PATH,
    get_hyperparameters,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
)

  0%|          | 0/18 [00:00<?, ?it/s]

[17:39:42] batch_size: 256 - nodes_per_feature: 0.5 - n_layers: 1 - activation: relu - dropout: -1 # already trained
[17:39:42] batch_size: 256 - nodes_per_feature: 0.5 - n_layers: 1 - activation: tanh - dropout: -1 # already trained
[17:39:42] batch_size: 256 - nodes_per_feature: 0.5 - n_layers: 2 - activation: relu - dropout: -1 # already trained
[17:39:42] batch_size: 256 - nodes_per_feature: 0.5 - n_layers: 2 - activation: tanh - dropout: -1 # already trained
[17:39:42] batch_size: 256 - nodes_per_feature: 0.5 - n_layers: 3 - activation: relu - dropout: -1 # already trained
[17:39:42] batch_size: 256 - nodes_per_feature: 0.5 - n_layers: 3 - activation: tanh - dropout: -1 # already trained
[17:39:42] batch_size: 256 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained
[17:39:42] batch_size: 256 - nodes_per_feature: 1 - n_layers: 1 - activation: tanh - dropout: -1 # already trained
[17:39:42] batch_size: 256 - nodes_per_feature: 1 - n_layers: 2 - ac

In [17]:
results = get_results_df(NN_SECOND_STAGE_RESULTS_PATH)
best_model = (
    results[
        (results["h3_res"] == TUNE_H3_RESOLUTION)
        & (results["time_interval_length"] == TUNE_TIME_INTERVAL_LENGTH)
    ]
    .sort_values(by="val_mape", ascending=True)
    .iloc[0]
)

best_config = {
    "batch_size": best_model["batch_size"],
    "nodes_per_feature": best_model["nodes_per_feature"],
    "n_layers": best_model["n_layers"],
    "activation": best_model["activation"],
}

best_config


{'batch_size': 256,
 'nodes_per_feature': 1.0,
 'n_layers': 1,
 'activation': 'relu'}

In [18]:
get_hyperparameters = lambda : get_third_stage_hyperparameters(
    best_batch_size=best_model["batch_size"],
    best_nodes_per_feature=best_model["nodes_per_feature"],
    best_n_layers=best_model["n_layers"],
    best_activation=best_model["activation"],
)
execute_stage(
    get_model_data,
    NN_THIRD_STAGE_RESULTS_PATH,
    get_hyperparameters,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
)


  0%|          | 0/5 [00:00<?, ?it/s]

[17:39:44] batch_size: 256 - nodes_per_feature: 1.0 - n_layers: 1 - activation: relu - dropout: 0 # already trained
[17:39:44] batch_size: 256 - nodes_per_feature: 1.0 - n_layers: 1 - activation: relu - dropout: 0.05 # already trained
[17:39:44] batch_size: 256 - nodes_per_feature: 1.0 - n_layers: 1 - activation: relu - dropout: 0.1 # already trained
[17:39:44] batch_size: 256 - nodes_per_feature: 1.0 - n_layers: 1 - activation: relu - dropout: 0.2 # already trained
[17:39:44] batch_size: 256 - nodes_per_feature: 1.0 - n_layers: 1 - activation: relu - dropout: 0.5 # already trained


In [19]:
results = get_results_df(NN_THIRD_STAGE_RESULTS_PATH)
best_dropout = (
	results[
		(results["h3_res"] == TUNE_H3_RESOLUTION)
		& (results["time_interval_length"] == TUNE_TIME_INTERVAL_LENGTH)
	]
	.sort_values(by="val_mape", ascending=True)["dropout"]
	.iloc[0]
)
best_config = {
	**best_config,
	"dropout": best_dropout,
}
best_config
		

{'batch_size': 256,
 'nodes_per_feature': 1.0,
 'n_layers': 1,
 'activation': 'relu',
 'dropout': 0.1}

In [20]:
for h3_res, time_interval_length in tqdm(list(product(PREDICTIVE_H3_RESOLUTIONS, CALC_TIME_INTERVAL_LENGTHS)) + ADDITIONAL_PREDICTIVE_RESOLUTIONS):
	execute_stage(
		get_model_data,
		NN_FOURTH_STAGE_RESULTS_PATH,
		lambda : [best_config],
		h3_res,
		time_interval_length,
		test_phase=True,
		silent=True,
	)

  0%|          | 0/9 [00:00<?, ?it/s]

In [48]:
results = get_results_df(NN_FOURTH_STAGE_RESULTS_PATH)
results

,h3_res,time_interval_length,batch_size,nodes_per_feature,n_layers,activation,dropout,test_mse,test_mae,test_mape,test_rmse
0,7,1,256,1.0,1,relu,0.1,4.612772,1.299698,0.568958,2.147736
1,7,2,256,1.0,1,relu,0.1,12.687275,2.107652,0.674838,3.561920
2,7,6,256,1.0,1,relu,0.1,69.654426,4.426815,0.799685,8.345923
3,7,24,256,1.0,1,relu,0.1,160.835376,6.272833,0.524323,12.682089
4,8,1,256,1.0,1,relu,0.1,0.267485,0.288976,0.243650,0.517189
5,8,2,256,1.0,1,relu,0.1,0.464892,0.417203,0.326760,0.681830
6,8,6,256,1.0,1,relu,0.1,1.313179,0.704363,0.458645,1.145940
7,8,24,256,1.0,1,relu,0.1,5.305457,1.359390,0.611528,2.303358
8,9,24,256,1.0,1,relu,0.1,1.639149,1.167411,1.000000,1.280292
9,77,24,256,1.0,1,relu,0.1,163.470526,6.228677,0.520632,12.785559


Now orig/dest

In [22]:
def read_orig_dest_data():
    model_data = pd.read_feather(
        os.path.join(
            MODEL_DATA_DIR_PATH,
            f"demand_{ORIGIN_DESTINATION_H3_RESOLUTION}_{ORIGIN_DESTINATION_TIME_INTERVAL_LENGTH}.feather",
        )
    )
    model_data = model_data.rename(columns={"demand": "outcome"})
    return model_data


In [23]:
model_data = read_orig_dest_data()

In [49]:
orig_dest_res_label = int(f"{ORIGIN_DESTINATION_H3_RESOLUTION}{ORIGIN_DESTINATION_H3_RESOLUTION}")

In [46]:
execute_stage(
    lambda _, __: model_data,
    NN_FOURTH_STAGE_RESULTS_PATH,
    lambda : [best_config],
    orig_dest_res_label,
    ORIGIN_DESTINATION_TIME_INTERVAL_LENGTH,
    test_phase=True,
    silent=True,
)

In [52]:
def get_demand_stats(row):
    h3_res, time_interval_length = row["h3_res"], row["time_interval_length"]

    model_data = (
        read_orig_dest_data()
        if h3_res == orig_dest_res_label
        else get_demand_model_data(h3_res, time_interval_length)
    )
    median = model_data.outcome.median()
    mean = model_data.outcome.mean()
    std = model_data.outcome.std()
    maximum = model_data.outcome.max()

    return pd.Series({"median": median, "mean": mean, "std": std, "maximum": maximum})


results = pd.concat(
    [
        results,
        results.apply(get_demand_stats, axis=1),
    ],
    axis=1,
)


In [51]:
results

,h3_res,time_interval_length,batch_size,nodes_per_feature,n_layers,activation,dropout,test_mse,test_mae,test_mape,test_rmse,median,mean,std,maximum
0,7,1,256,1.0,1,relu,0.1,4.612772,1.299698,0.568958,2.147736,1.0,2.275141,2.348778,44.0
1,7,2,256,1.0,1,relu,0.1,12.687275,2.107652,0.674838,3.561920,2.0,3.107402,3.935554,78.0
2,7,6,256,1.0,1,relu,0.1,69.654426,4.426815,0.799685,8.345923,2.0,5.468982,9.092437,171.0
3,7,24,256,1.0,1,relu,0.1,160.835376,6.272833,0.524323,12.682089,3.0,12.146287,23.857396,343.0
4,8,1,256,1.0,1,relu,0.1,0.267485,0.288976,0.243650,0.517189,1.0,1.186809,0.525155,13.0
5,8,2,256,1.0,1,relu,0.1,0.464892,0.417203,0.326760,0.681830,1.0,1.278349,0.701289,21.0
6,8,6,256,1.0,1,relu,0.1,1.313179,0.704363,0.458645,1.145940,1.0,1.538647,1.216924,37.0
7,8,24,256,1.0,1,relu,0.1,5.305457,1.359390,0.611528,2.303358,1.0,2.207834,2.424620,65.0
8,9,24,256,1.0,1,relu,0.1,1.639149,1.167411,1.000000,1.280292,1.0,1.166299,0.524977,22.0
9,77,24,256,1.0,1,relu,0.1,163.470526,6.228677,0.520632,12.785559,3.0,12.146287,23.857396,343.0
